# 리소스 클린업

# 1. 환경 설정

In [1]:
%store -r repository_name
%store -r code_build_project_name
%store -r sm_pipeline_name
%store -r code_pipeline_name
%store -r bucket

# 2. 코드 리파지토리 삭제

In [2]:
%%sh -s {repository_name}
repository_name=$1
aws codecommit delete-repository --repository-name $repository_name 


{
    "repositoryId": "aba40d31-1cdb-48bb-9829-81c699f969eb"
}


# 3. 코드 빌드 프로젝트 삭제

In [3]:
%%sh -s {code_build_project_name}
code_build_project_name=$1
aws codebuild delete-project --name $code_build_project_name

# 4. 코드 파이프라인 삭제

In [4]:
%%sh -s {code_pipeline_name}
pipeline_name=$1
aws codepipeline delete-pipeline --name $pipeline_name

# 5. SageMaker Model Building Pipeline 삭제




## 5.1 파이프라인 삭제

- 파이프라인 이름 검색

sm_pipeline_name 의 변수 이름으로 시작되는 파이프라인 정보 검색

In [5]:
import boto3

client = boto3.client('sagemaker')

response = client.list_pipelines(
    PipelineNamePrefix= sm_pipeline_name,
    SortBy='Name',
    SortOrder='Descending',
    MaxResults=1
)

response

{'PipelineSummaries': [{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/ncf-sm-pipeline',
   'PipelineName': 'ncf-sm-pipeline',
   'PipelineDisplayName': 'ncf-sm-pipeline',
   'RoleArn': 'arn:aws:iam::057716757052:role/codebuild-gsmoon',
   'CreationTime': datetime.datetime(2022, 7, 8, 12, 23, 50, 696000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2022, 7, 8, 12, 35, 53, 790000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': 'dae9c56a-49ad-4701-80f3-8ad36d428de2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dae9c56a-49ad-4701-80f3-8ad36d428de2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '307',
   'date': 'Fri, 08 Jul 2022 12:52:21 GMT'},
  'RetryAttempts': 0}}

In [6]:
response = client.delete_pipeline(
    PipelineName=sm_pipeline_name,
)
response

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/ncf-sm-pipeline',
 'ResponseMetadata': {'RequestId': 'c20ed1b6-284b-4b10-b35e-2fd0aa12c226',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c20ed1b6-284b-4b10-b35e-2fd0aa12c226',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '83',
   'date': 'Fri, 08 Jul 2022 12:52:21 GMT'},
  'RetryAttempts': 0}}

## 5.2. 파이프라인 관련 S3 내 파일 삭제

In [7]:
# ! aws s3 ls {bucket}/{sm_pipeline_name} --recursive
! aws s3 rm s3://{bucket}/{sm_pipeline_name} --recursive

# 6. 모델 레지스트리 삭제

- 자세한 것은 개발자 가이드 참고 하세요. [Boto3 SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.delete_model_package)

## 6.1. 전체 model package group 검색
- NameContains 의 이름 있는 것만 검색

In [8]:
response = client.list_model_package_groups(
    NameContains = 'mimick',
    SortBy='CreationTime',
    SortOrder='Ascending'
)
response

{'ModelPackageGroupSummaryList': [],
 'NextToken': 'cIws2QhTXUIa8bi8XY6Q7JUM+leXzY+WUYv213SjXJg6fOXnrUzo74zCATuP2us05VS7xlmHZoFNMQIKzAy/+cwXW0dvNpjXuCsswXlIVdlWe5c1IaKt/FsGFwKJog1DXbQyE28kboi77xb3z8o3hY5I7Pgdh93ZHHCIrIqyv1aAFi3G45swZdoRqXF5ZYcZPa+lM8JhEyweI8lLRTD6DSnGvwsT9J3w0VePy2QoR6hCNAlKMKYQ+LXF6dJ54ybTeOT7BPpGA/s3DDVdUoMOoeDYdZ+g/BUi24YupoD7hTzcgSEElWucWt4UoNmOPt8+QYJk0JwkHyx2VD5+JQ5aApEHXMOMgD/d6u03ahDZ/bLVBFNGw0Rswoe4TcVacUMQLBcTUa33cq162yN26XhLzog02bBsktzDfv4CGI/Gz+LP2mJJdiX9tr40VajWYEYmC0VS3y/AVTAzlN+FAiXvFkO3M35LFGrkz8AeEAY4OQrDcU8y0+566mPq/OxBVVX7/05sxKmsuPFaBh7K8NnkP8mhD/YW4nuyRcktTQEOinNcWK6h1YotNNXLASjAbYgNL0jlhmiBh+gWtBmnRA==',
 'ResponseMetadata': {'RequestId': '8f14e831-1b58-43ce-87a6-22e34f152fd7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8f14e831-1b58-43ce-87a6-22e34f152fd7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '646',
   'date': 'Fri, 08 Jul 2022 12:52:21 GMT'},
  'RetryAttempts': 0}}

## 6.2. 기술된 model package group 안애서 등록된 model package 만 검색

In [9]:
model_package_group_name = sm_pipeline_name

In [10]:
response = client.list_model_packages(
    ModelPackageGroupName=model_package_group_name , 
    SortBy='CreationTime',
    SortOrder='Ascending'
)
response

{'ModelPackageSummaryList': [],
 'ResponseMetadata': {'RequestId': '3a4771c5-c5cf-471d-b6df-cf5ef37ead34',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3a4771c5-c5cf-471d-b6df-cf5ef37ead34',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '30',
   'date': 'Fri, 08 Jul 2022 12:52:22 GMT'},
  'RetryAttempts': 0}}

## 6.3. model package 삭제

In [11]:
for e in response['ModelPackageSummaryList']:
    print(e)
    ModelPackageArn = e['ModelPackageArn']
    print(f"########## Delete: {ModelPackageArn}")
    response = client.delete_model_package(
        ModelPackageName=ModelPackageArn
    )    
    print(response)

## 6.4. 기술된 model package group 삭제

In [12]:
response = client.delete_model_package_group(
    ModelPackageGroupName= model_package_group_name
)
response

ClientError: An error occurred (ValidationException) when calling the DeleteModelPackageGroup operation: ModelPackageGroup arn:aws:sagemaker:us-east-1:057716757052:model-package-group/ncf-sm-pipeline does not exist.